In [ ]:
! pip install verde harmonica pandas matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import harmonica as hm
import verde as vd
import pandas as pd
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
region = [0, 100e3, 0, 90e3]
coordinates = vd.scatter_points(region, size=1000, extra_coords=1000)

plt.plot(coordinates[0], coordinates[1], ".k")

In [ ]:
len(coordinates[0])

In [ ]:
prismas = [
    [10e3, 15e3, 20e3, 35e3, -1e3,  0],
    [60e3, 80e3, 50e3, 70e3, -2e3,  -0.5e3],
]
densidades = [500, -300]
gz = hm.prism_gravity(coordinates, prismas, densidades, field="g_z")

In [ ]:
plt.scatter(coordinates[0], coordinates[1], s=5, c=gz)
plt.colorbar()

In [ ]:
# Fontes equivalentes
fontes = hm.EquivalentSourcesGB(damping=1,depth=1e3)
altura = np.zeros(1000)
fontes.fit([coordinates[0],coordinates[1],altura],data=gz) # o que faz essa funcao .fit?

In [ ]:
grid_coords_eq = vd.grid_coordinates(region,spacing=1e3,extra_coords=1000)
gz_grid_eq = fontes.grid(grid_coords_eq,data_names="g_z")
gz_grid_eq.g_z.plot()

In [ ]:
# Malha regular
grid_coords = vd.grid_coordinates(region, spacing=1e3, extra_coords=1000)
gz_grid = hm.prism_gravity(grid_coords, prismas, densidades, field="g_z")
grid_true = vd.make_xarray_grid(grid_coords, gz_grid, data_names="g_z", extra_coords_names="upward")
grid_true.g_z.plot()


In [ ]:
##################################### Diferença
residuo = grid_true - gz_grid_eq
residuo.g_z.plot()

In [ ]:
residuo.g_z

In [ ]:
gz_grid_eq.g_z

In [ ]:
rms = np.sum([res**2 for res in residuo.g_z])/residuo.g_z.size
##### R_2 verdadeiro
media_grid_eq = np.sum(gz_grid_eq.g_z.values)
R_2_true = 1 - (np.sum([res**2 for res in residuo.g_z])/np.sum([( gz - media_grid_eq )**2 for gz in gz_grid_eq.g_z]))
R_2_true

In [ ]:
# Using cross-validation to evaluate how well
# these equivalent sources can accurately predict the values of the field
R_2_kfold = (
    vd.cross_val_score(
        fontes,
        coordinates,
        gz,
    )
)


In [ ]:
print(f"min: {np.min(R_2_kfold)}")
print(f"max: {np.max(R_2_kfold)}")
print(f"mean: {np.mean(R_2_kfold)}")

In [ ]:
# Blocking

R_2_block = (
    vd.cross_val_score(
        fontes,
        coordinates,
        gz,
        cv=vd.BlockKFold(spacing=500, n_splits=2, shuffle=True, random_state=123),
    )
)


In [ ]:
print(f"min: {np.min(R_2_block)}")
print(f"max: {np.max(R_2_block)}")
print(f"mean: {np.mean(R_2_block)}")

In [ ]:
#Data are first grouped into rectangular blocks of size given by the spacing argument.
# The blocks are then split into testing and training sets iteratively along k folds of the data
#(k is given by n_splits).
R_2_block = []
n = np.arange(2,51)
for n_splits in n:
    R_2_block.append(np.mean(
        vd.cross_val_score(
            fontes,
            coordinates,
            gz,
            cv=vd.BlockKFold(spacing=500, n_splits=n_splits, shuffle=True, random_state=123),
        )
    ))



In [ ]:
plt.plot(n,R_2_block)
plt.legend()
plt.xlabel("n_splits")
plt.ylabel("R_2")

In [ ]:
R_2_block = []
block = np.arange(500,10950,950)
for spacing in block:
    R_2_block.append(np.mean(
        vd.cross_val_score(f
            fontes,
            coordinates,
            gz,
            cv=vd.BlockKFold(spacing=spacing, n_splits=4, shuffle=True, random_state=123),
        )
    ))



In [ ]:
plt.plot(block,R_2_block)
plt.legend()
plt.xlabel("Tamanho do bloco")
plt.ylabel("R_2")

In [ ]:
plt.plot(block,R_2_block)
plt.legend()
plt.xlabel("Tamanho do bloco")
plt.ylabel("R_2")
plt.xlim(0,4000)

In [ ]:
##################################### Colocando blocos nas fontes equivalentes
R_2 = []
sources = []

block = [block for block in range(50,102050,2000)]
for block_size in block:
    fontes_block = hm.EquivalentSourcesGB(damping=1,depth=1e3,block_size=block_size, depth_type="constant")
    # These sources were set at a constant depth of 1km bellow the zeroth height and with a damping equal to 1.

    fontes_block.fit([coordinates[0],coordinates[1],altura],data=gz)
    # During this step the point sources are created through the block averaging process.

    # Using cross-validation to evaluate how well
    # these equivalent sources can accurately predict the values of the field
    score = np.mean(
        vd.cross_val_score(
            fontes_block,
            coordinates,
            gz,
        )
    )
    R_2.append(score)
    sources.append(fontes_block.points_[0].size)


In [ ]:
plt.plot(block,R_2)
plt.legend()
plt.xlabel('Tamanho do bloco')
plt.ylabel('Erro')

In [ ]:
plt.plot(block,sources)
plt.legend()
plt.xlabel('Tamanho do bloco')
plt.ylabel('Numero de fontes')

In [ ]:
plt.plot(sources,R_2)
plt.legend()
plt.xlabel('Numero de fontes')
plt.ylabel('Erro')

In [ ]:
plt.plot(sources,R_2)
plt.legend()
plt.xlabel('Numero de fontes')
plt.xlim(-3,150)
plt.ylabel('Erro')